In [ ]:
!unzip brain_tumor_dataset.zip

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torch.optim import AdamW
from torchvision.datasets import ImageFolder
from transformers import CLIPImageProcessor, CLIPVisionModel
from tqdm import tqdm

In [ ]:
preprocessor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
vit_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32").cuda()

In [ ]:
def preprocess_image(img):
    return preprocessor(img, return_tensors="pt")["pixel_values"][0]

In [ ]:
dataset = ImageFolder("brain_tumor_dataset", transform=preprocess_image)
train_dataset, test_dataset = random_split(dataset, (0.9, 0.1))
print(f"{len(train_dataset)} training samples")
print(f"{len(test_dataset)} training samples")

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model = nn.Sequential(
    nn.Dropout(0.1),
    nn.Linear(768, 2),
).cuda()
optimizer = AdamW(model.parameters(), lr=1e-3)

In [ ]:
model.train()
pbar = tqdm(train_dataloader)
for batch in pbar:
    x, y = batch
    optimizer.zero_grad()
    pred = model(vit_model(x.cuda()).pooler_output)
    loss = F.cross_entropy(pred, y.cuda())
    loss.backward()
    optimizer.step()
    pbar.set_description(f"Loss: {loss.item():.4f}")

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for x, y in test_dataloader:
        y_pred = model(vit_model(x.cuda()).pooler_output)
        correct += (y_pred.argmax(dim=1) == y.cuda()).sum().item()
        total += len(y)
    accuracy = correct / total
print(accuracy)